<a href="https://colab.research.google.com/github/munazza-a11/GANS-FYP-/blob/ivm/Set_up.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!git clone https://github.com/munazza-a11/GANS-FYP-.git


Cloning into 'GANS-FYP-'...
remote: Enumerating objects: 49, done.
remote: Counting objects: 100% (49/49), done.
remote: Compressing objects: 100% (40/40), done.
remote: Total 49 (delta 10), reused 45 (delta 7), pack-reused 0 (from 0)
Receiving objects: 100% (49/49), 2.71 MiB | 14.38 MiB/s, done.
Resolving deltas: 100% (10/10), done.


In [ ]:
%cd GANS-FYP-


/content/GANS-FYP-


In [ ]:
!git checkout -b sprint2-ivm-core

Switched to a new branch 'sprint2-ivm-core'


In [ ]:
!nvidia-smi
import torch
print("CUDA Available:", torch.cuda.is_available())

Fri Nov  7 14:03:19 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   57C    P8             10W /   70W |       0MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [ ]:
!pip install ultralytics

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 33.0 MB/s eta 0:00:00


In [ ]:
import torch
import torchvision.models as models
from torchvision.models import efficientnet_b0, EfficientNet_B0_Weights
from torchvision import transforms
from ultralytics import YOLO
from PIL import Image


Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"

# Load EfficientNet-B0 pretrained on ImageNet
weights = EfficientNet_B0_Weights.IMAGENET1K_V1
efficientnet = efficientnet_b0(weights=weights).to(device)
efficientnet.eval()

# Load YOLO
yolo = YOLO("yolov5s.pt")


Downloading: "https://download.pytorch.org/models/efficientnet_b0_rwightman-7f5810bc.pth" to /root/.cache/torch/hub/checkpoints/efficientnet_b0_rwightman-7f5810bc.pth


100%|██████████| 20.5M/20.5M [00:00<00:00, 151MB/s]


PRO TIP 💡 Replace 'model=yolov5s.pt' with new 'model=yolov5su.pt'.
YOLOv5 'u' models are trained with https://github.com/ultralytics/ultralytics and feature improved performance vs standard YOLOv5 models trained with https://github.com/ultralytics/yolov5.



In [ ]:
preprocess = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
])

def prepare_image(path):
    img = Image.open(path).convert("RGB")
    return preprocess(img).unsqueeze(0).to(device)


In [ ]:
def run_models(image_path):
    x = prepare_image(image_path)

    # EfficientNet
    with torch.no_grad():
        logits = efficientnet(x)
        probs = torch.softmax(logits[0], dim=0)
        effnet_conf = torch.max(probs).item()

    # YOLO
    yolo_out = yolo(image_path)
    yolo_conf = max([box.conf for box in yolo_out[0].boxes], default=0)

    return effnet_conf, yolo_conf


In [ ]:
def consensus(effnet_conf, yolo_conf, threshold=0.3):
    if abs(effnet_conf - yolo_conf) > threshold:
        return "⚠️ Potential Adversarial Attack"
    else:
        return "✅ Normal Image"


In [ ]:
image = "/content/pexels-cottonbro-studio-5473956-min_11zon.jpg"
e_conf, y_conf = run_models(image)
print("EfficientNet Confidence:", e_conf)
print("YOLO Confidence:", y_conf)
print("Decision:", consensus(e_conf, y_conf))


image 1/1 /content/pexels-cottonbro-studio-5473956-min_11zon.jpg: 448x640 1 person, 43.3ms
Speed: 13.5ms preprocess, 43.3ms inference, 38.3ms postprocess per image at shape (1, 3, 448, 640)
EfficientNet Confidence: 0.2054690420627594
YOLO Confidence: tensor([0.7061], device='cuda:0')
Decision: ⚠️ Potential Adversarial Attack


In [ ]:
image = "/content/british-wild-cat-1.jpg"
e_conf, y_conf = run_models(image)
print("EfficientNet Confidence:", e_conf)
print("YOLO Confidence:", y_conf)
print("Decision:", consensus(e_conf, y_conf))


image 1/1 /content/british-wild-cat-1.jpg: 384x640 2 cats, 44.1ms
Speed: 2.1ms preprocess, 44.1ms inference, 1.7ms postprocess per image at shape (1, 3, 384, 640)
EfficientNet Confidence: 0.5520377159118652
YOLO Confidence: tensor([0.5177], device='cuda:0')
Decision: ✅ Normal Image
